<a href="https://colab.research.google.com/github/gnunes-io/etl_vinicula/blob/main/ETL_FiapChall.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 📚 Importando bibliotecas

In [2]:
import requests

import pandas as pd

import glob

import os

import warnings
warnings.filterwarnings('ignore')

from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


# 🍇 Coleta e Transformação de dados (Dados vinícula)

###🔹 Dados de Processamento

In [ ]:
# Importando processamento.csv
processamento = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Vinícula/Processamento.csv', sep=';', encoding='utf-8', decimal=',')

In [ ]:
processamento.head(5)

,id,control,cultivar,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,TINTAS,TINTAS,10448228,11012833,10798824,8213674,17457849,22593885,20265190,...,29810706,29935627,13370866,32850915,26868514,nd,28003505,93296587,*,35881118.23
1,2,ti_Alicante Bouschet,Alicante Bouschet,0,0,0,0,0,0,0,...,1456305,1519576,908841,2040198,2103844,nd,2272985,811140,*,4108858.21
2,3,ti_Ancelota,Ancelota,0,0,0,0,0,0,0,...,937844,773526,179028,733907,492106,nd,481402,6513974,*,783688.39
3,4,ti_Aramon,Aramon,0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,0,*,0.00
4,5,ti_Alfrocheiro,Alfrocheiro,0,0,0,0,0,0,0,...,0,0,0,0,0,nd,0,0,*,0.00


In [ ]:
def transform_processamento(processamento):

    # Transformando colunas em linhas
    processamento = processamento.melt(
        id_vars=['id', 'control', 'cultivar'],
        var_name='Ano',
        value_name='Quantidade'
    )

    # Limpando dados da coluna Quantidade
    processamento['Quantidade'] = (
        processamento['Quantidade']
        .replace(['*', 'nd', ''], 0)
        .astype(float)
    )

    # Removendo registros de Quantidade = zero (se necessário)
    processamento = processamento[processamento['Quantidade'] != 0]

    # Padronizando nome da cultivar
    processamento['cultivar'] = processamento['cultivar'].str.strip().str.title()

    return processamento

# Aplicando transformação
processamento = transform_processamento(processamento)

# Definindo prefixos para mapear grupos
prefixos = {
    'ti_': 'TINTAS',
    'br_': 'BRANCAS E ROSADAS'
}

# Criando função para mapear grupo a partir de controle
def mapear_grupo(control):
    control_lower = control.lower()
    for pref, nome in prefixos.items():
        if control_lower.startswith(pref):
            return nome
    return 'Outro'

# Atribuindo grupo aos registros
processamento['grupo'] = processamento['control'].apply(mapear_grupo)

# Filtrando grupos válidos (não subtotais)
processamento = processamento[processamento['grupo'] != "Outro"].reset_index(drop=True)

# Aplicando política de tipagem
processamento = processamento.astype({
    'id': 'string',
    'control': 'string',
    'cultivar': 'string',
    'Ano': 'int64',
    'Quantidade': 'float',
    'grupo': 'category'
})

# Renomeando colunas
processamento.rename(columns={
    'id': 'Id',
    'control': 'Controle',
    'cultivar': 'Produto',
    'grupo': 'Grupo'
}, inplace=True)

processamento

## Existe uma diferença de 20kg entre o subtotal e as linhas de detalhe no ano de 2020. [ERRO]

,Id,Controle,Produto,Ano,Quantidade,Grupo
0,8,ti_Barbera,Barbera,1970,4548313.00,TINTAS
1,9,ti_Bonarda,Bonarda,1970,1631610.00,TINTAS
2,10,ti_Cabernet Franc,Cabernet Franc,1970,238837.00,TINTAS
3,14,ti_Canaiolo,Canaiolo,1970,596766.00,TINTAS
4,16,ti_Carmenere,Carmenere,1970,1235.00,TINTAS
...,...,...,...,...,...,...
2915,133,br_Verdelho,Verdelho,2023,2080.00,BRANCAS E ROSADAS
2916,135,br_Vermentino,Vermentino,2023,28218.00,BRANCAS E ROSADAS
2917,136,br_Vernaccia,Vernaccia,2023,800.00,BRANCAS E ROSADAS
2918,137,br_Viogner,Viogner,2023,21794.25,BRANCAS E ROSADAS


In [ ]:
# Exportando para o Gold
processamento.to_excel('/content/drive/MyDrive/Challenge_FIAP/Gold_Data/processamento.xlsx', index=False)
processamento.to_parquet('/content/drive/MyDrive/Challenge_FIAP/Gold_Data/processamento.parquet', index=False)

###🔹 Dados de Comercialização

In [ ]:
# Importando comercializacao.csv
comercializacao = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Vinícula/Comercializacao.csv', sep=';', encoding='utf-8', decimal=',')

In [ ]:
def transform_comercializacao(comercializacao):

    # Transformando colunas em linhas
    comercializacao = comercializacao.melt(
        id_vars=['id', 'control', 'Produto'],
        var_name='Ano',
        value_name='Quantidade'
    )

    # Limpando dados da coluna Quantidade
    comercializacao['Quantidade'] = (
        comercializacao['Quantidade']
        .replace(['*', 'nd', ''], 0)
        .astype(float)
    )

    # Removendo registros com Quantidade zero (se necessário)
    comercializacao = comercializacao[comercializacao['Quantidade'] != 0]

    # Padronizando nome do Produto
    comercializacao['Produto'] = comercializacao['Produto'].str.strip().str.title()

    return comercializacao

# Aplicando transformação
comercializacao = transform_comercializacao(comercializacao)

# Definindo prefixos para mapear grupos
prefixos = {
    'vinho frizante': 'VINHO FRIZANTE',
    'vinho orgânico': 'VINHO ORGÂNICO',
    've_': 'VINHO ESPECIAL',
    'es_': 'ESPUMANTES',
    'su_': 'SUCO DE UVAS',
    'suco de uvas concentrado': 'SUCO DE UVAS CONCENTRADO',
    'ou_': 'OUTROS PRODUTOS COMERCIALIZADOS'
}

# Criando função para mapear grupo a partir de controle
def mapear_grupo(control):
    # Retornando 'Outro' se valor for nulo
    if pd.isna(control):
        return 'Outro'
    control_lower = str(control).lower()
    for pref, nome in prefixos.items():
        if control_lower.startswith(pref):
            return nome
    return 'Outro'

# Atribuindo grupo aos registros
comercializacao['grupo'] = comercializacao['control'].apply(mapear_grupo)

# Definindo mapa para IDs específicos
mapa_grupos = {
    (2, 3, 4): 'VINHO DE MESA',
    (6, 7, 8): 'VINHO FINO DE MESA'
}

# Atualizando grupo para IDs específicos
for ids, grupo in mapa_grupos.items():
    comercializacao.loc[comercializacao['id'].isin(ids), 'grupo'] = grupo

# Filtrando apenas grupos válidos
comercializacao = comercializacao[comercializacao['grupo'] != "Outro"].reset_index(drop=True)

# Aplicando política de tipagem
comercializacao = comercializacao.astype({
    'id': 'string',
    'control': 'string',
    'Produto': 'string',
    'Ano': 'int64',
    'Quantidade': 'float',
    'grupo': 'category'
})

# Renomeando colunas
comercializacao.rename(columns={
    'id': 'Id',
    'control': 'Controle',
    'grupo': 'Grupo'
}, inplace=True)

comercializacao

,Id,Controle,Produto,Ano,Quantidade,Grupo
0,2,vm_Tinto,Tinto,1970,83300735.0,VINHO DE MESA
1,3,vm_Rosado,Rosado,1970,107681.0,VINHO DE MESA
2,4,vm_Branco,Branco,1970,14919190.0,VINHO DE MESA
3,6,vm_Tinto,Tinto,1970,435354.0,VINHO FINO DE MESA
4,7,vm_Rosado,Rosado,1970,183234.0,VINHO FINO DE MESA
...,...,...,...,...,...,...
1514,57,ou_Vinho_acetificado,Vinho Acetificado,2023,194020.0,OUTROS PRODUTOS COMERCIALIZADOS
1515,59,ou_Vinho_composto,Vinho Composto,2023,981.0,OUTROS PRODUTOS COMERCIALIZADOS
1516,60,ou_Vinho_licoroso,Vinho Licoroso,2023,421974.0,OUTROS PRODUTOS COMERCIALIZADOS
1517,61,ou_Vinho_leve,Vinho Leve,2023,132064.0,OUTROS PRODUTOS COMERCIALIZADOS


###🔹 Dados de Produção

In [ ]:
# Importando producao.csv
producao = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Vinícula/Producao.csv', sep=';', encoding='utf-8', decimal=',')

In [ ]:
producao.head()

,id,control,produto,1970,1971,1972,1973,1974,1975,1976,...,2014,2015,2016,2017,2018,2019,2020,2021,2022,2023
0,1,VINHO DE MESA,VINHO DE MESA,217208604,154264651,146953297,116710345,193875345,177401209,144565438,...,196173123,210308560,86319015,255015187,218375636,144629737,124200414,173899995,195031611,169762429
1,2,vm_Tinto,Tinto,174224052,121133369,118180926,88589019,146544484,144274134,118360170,...,157776363,169811472,75279191,1365957,188270142,121045115,103916391,146075996,162844214,139320884
2,3,vm_Branco,Branco,748400,1160500,1812367,243900,4138768,1441507,1871473,...,37438069,39557250,10727099,217527985,29229970,22032828,19568734,26432799,30198430,27910299
3,4,vm_Rosado,Rosado,42236152,31970782,26960004,27877426,43192093,31685568,24333795,...,958691,939838,312725,36121245,875524,1551794,715289,1391200,1988968,2531246
4,5,VINHO FINO DE MESA (VINIFERA),VINHO FINO DE MESA (VINIFERA),23899346,23586062,21078771,12368410,31644124,39424590,34500590,...,38464314,37148982,18070626,44537870,38707220,37615422,32516686,43474998,47511796,46268556


In [ ]:
# Definindo função para transformar tabela de produção
def transform_producao(producao):

    # Transformando colunas em linhas
    producao = producao.melt(
        id_vars=['id', 'control', 'produto'],
        var_name='Ano',
        value_name='Quantidade'
    )

    # Limpando dados da coluna Quantidade
    producao['Quantidade'] = (
        producao['Quantidade']
        .replace(['*', 'nd', ''], 0)
        .astype(float)
    )

    # Removendo registros com Quantidade zero (se necessário)
    producao = producao[producao['Quantidade'] != 0]

    # Padronizando nome do Produto
    producao['produto'] = producao['produto'].str.strip().str.title()

    return producao

# Aplicando transformação
producao = transform_producao(producao)

# Definindo prefixos para mapear grupos
prefixos = {
    'vm_': 'VINHO DE MESA',
    'vv_': 'VINHO FINO DE MESA (VINIFERA)',
    'su_': 'SUCO',
    'de_': 'DERIVADOS'
}

# Criando função para mapear grupo a partir de controle
def mapear_grupo(control):
    control_lower = control.lower()
    for pref, nome in prefixos.items():
        if control_lower.startswith(pref):
            return nome
    return 'Outro'

# Atribuindo grupo aos registros
producao['grupo'] = producao['control'].apply(mapear_grupo)

# Filtrando apenas grupos válidos
producao = producao[producao['grupo'] != "Outro"].reset_index(drop=True)

# Aplicando política de tipagem
producao = producao.astype({
    'id': 'string',
    'control': 'string',
    'produto': 'string',
    'Ano': 'int64',
    'Quantidade': 'float',
    'grupo': 'category'
})

# Renomeando colunas
producao.rename(columns={
    'id': 'Id',
    'control': 'Controle',
    'produto': 'Produto',
    'grupo': 'Grupo'
}, inplace=True)

producao

,Id,Controle,Produto,Ano,Quantidade,Grupo
0,2,vm_Tinto,Tinto,1970,174224052.0,VINHO DE MESA
1,3,vm_Branco,Branco,1970,748400.0,VINHO DE MESA
2,4,vm_Rosado,Rosado,1970,42236152.0,VINHO DE MESA
3,6,vv_Tinto,Tinto,1970,7591557.0,VINHO FINO DE MESA (VINIFERA)
4,7,vv_Branco,Branco,1970,15562889.0,VINHO FINO DE MESA (VINIFERA)
...,...,...,...,...,...,...
891,43,de_Borra seca,Borra Seca,2023,17200.0,DERIVADOS
892,46,de_Vinho orgânico,Vinho Orgânico,2023,94150.0,DERIVADOS
893,47,de_Espumante orgânico,Espumante Orgânico,2023,1365.0,DERIVADOS
894,49,de_Vinho acidificado,Vinho Acidificado,2023,2500.0,DERIVADOS


###🔹  Dados de Importação

In [ ]:
# Importando importacao.csv
importacao = pd.read_csv('/content/drive/MyDrive/Multimedia/Tech Challenge - 10DTAT/Importacao.csv', sep=';', encoding='utf-8')

In [ ]:
importacao.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Africa do Sul,0,0.0,0,0,0,0,0,0,...,1092042,3604038,627150,1701072,859169,2508140,738116,2266827,522733,1732850
1,2,Alemanha,52297,30498.0,34606,26027,134438,92103,111523,98638,...,101055,412794,136992,504168,106541,546967,92600,438595,102456,557947
2,3,Argélia,0,0.0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,4,Arábia Saudita,0,0.0,0,0,0,0,0,0,...,0,0,0,0,2510,8761,0,0,8,161
4,5,Argentina,19525,12260.0,24942,15022,104906,58137,116887,76121,...,16548931,54527380,22610267,66322932,26869241,79527959,27980574,87519642,25276991,83918138


In [ ]:
# Transformando colunas em linhas
importacao = importacao.melt(
    id_vars=['Id', 'País'],
    var_name='Ano',
    value_name='Valor'
)

# Criando coluna de tipo (KG ou USD)
importacao['Tipo'] = importacao['Ano'].astype(str).str.contains(r'\.1$')
importacao['Tipo'] = importacao['Tipo'].map({False: 'KG', True: 'USD'})

# Limpando o ano
importacao['Ano'] = importacao['Ano'].astype(str).str.replace(r'\.1$', '', regex=True)

# Aplicando política de tipagem
importacao = importacao.astype({
    'Id': 'string',
    'País': 'string',
    'Ano': 'int64',
    'Valor': 'float'
})

# Pivotando para abrir colunas KG e USD
importacao = importacao.pivot_table(
    index=['Id', 'País', 'Ano'],
    columns='Tipo',
    values='Valor',
    aggfunc='sum'
).reset_index()

# Ajustando o nome das colunas
importacao.columns.name = None

importacao


,Id,País,Ano,KG,USD
0,1,Africa do Sul,1970,0.0,0.0
1,1,Africa do Sul,1971,0.0,0.0
2,1,Africa do Sul,1972,0.0,0.0
3,1,Africa do Sul,1973,0.0,0.0
4,1,Africa do Sul,1974,0.0,0.0
...,...,...,...,...,...
3667,9,Bermudas,2019,0.0,0.0
3668,9,Bermudas,2020,0.0,0.0
3669,9,Bermudas,2021,10.0,141.0
3670,9,Bermudas,2022,1.0,4.0


###🔹 Dados de Exportacao

In [ ]:
# Importando exportacao.csv
exportacao = pd.read_csv('/content/drive/MyDrive/Multimedia/Tech Challenge - 10DTAT/Exportacao.csv', sep=';', encoding='utf-8')

In [ ]:
exportacao.head()

,Id,País,1970,1970.1,1971,1971.1,1972,1972.1,1973,1973.1,...,2019,2019.1,2020,2020.1,2021,2021.1,2022,2022.1,2023,2023.1
0,1,Afeganistão,0,0,0,0,0,0,0,0,...,0,0,0,0,11,46,0,0,0,0
1,2,África do Sul,0,0,0,0,0,0,0,0,...,26,95,4,21,0,0,0,0,117,698
2,3,"Alemanha, República Democrática",0,0,0,0,4168,2630,12000,8250,...,3660,25467,6261,32605,2698,6741,7630,45367,4806,31853
3,4,Angola,0,0,0,0,0,0,0,0,...,345,1065,0,0,0,0,4068,4761,0,0
4,5,Anguilla,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [ ]:
# Transformando colunas em linhas
exportacao = exportacao.melt(
    id_vars=['Id', 'País'],
    var_name='Ano',
    value_name='Valor'
)

# Cria coluna tipo (KG ou USD)
exportacao['Tipo'] = exportacao['Ano'].astype(str).str.contains(r'\.1$')
exportacao['Tipo'] = exportacao['Tipo'].map({False: 'KG', True: 'USD'})

# Limpando o ano (remove .1 via regex)
exportacao['Ano'] = exportacao['Ano'].astype(str).str.replace(r'\.1$', '', regex=True)

# Aplicando política de tipagem
exportacao = exportacao.astype({
    'Id': 'string',
    'País': 'string',
    'Ano': 'int64',
    'Valor': 'float'
})

# Pivotando para abrir colunas KG e USD
exportacao = exportacao.pivot_table(
    index=['Id', 'País', 'Ano'],
    columns='Tipo',
    values='Valor',
    aggfunc='sum'
).reset_index()

# Ajustando o nome das colunas
exportacao.columns.name = None

exportacao.head()

,Id,País,Ano,KG,USD
0,1,Afeganistão,1970,0.0,0.0
1,1,Afeganistão,1971,0.0,0.0
2,1,Afeganistão,1972,0.0,0.0
3,1,Afeganistão,1973,0.0,0.0
4,1,Afeganistão,1974,0.0,0.0


# 🌐 Coleta e Transformação de dados (Dados externos)

###🔹 Dados Climáticos

In [ ]:
# Consolidando todos os arquivos .csv em um
pasta = r"/content/drive/MyDrive/FIAP/Dados Temperatura"

# Definindo colunas para ignorar
colunas_remover = [
    'PRESSÃO ATMOSFERICA MAX.NA HORA ANT. (AUT) (mB)',
    'PRESSÃO ATMOSFERICA MIN. NA HORA ANT. (AUT) (mB)',
    'TEMPERATURA MÁXIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA MÍNIMA NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MAX. NA HORA ANT. (AUT) (°C)',
    'TEMPERATURA ORVALHO MIN. NA HORA ANT. (AUT) (°C)',
    'UMIDADE REL. MAX. NA HORA ANT. (AUT) (%)',
    'UMIDADE REL. MIN. NA HORA ANT. (AUT) (%)',
    'VENTO, DIREÇÃO HORARIA (gr) (° (gr))',
    'VENTO, RAJADA MAXIMA (m/s)'
]

dfs = []

for arquivo in glob.glob(os.path.join(pasta, "*.CSV")):
    # Lendo só as primeiras 5 linhas para pegar a estacao (cidade)
    header_temp = pd.read_csv(arquivo, sep=';', header=None, skip_blank_lines=False,
                              encoding='latin-1', nrows=5)
    estacao = header_temp.iloc[2, 1]

    # Lendo dados reais (pula 8 linhas)
    df = pd.read_csv(arquivo, sep=';', skiprows=8, encoding='utf-8')

    # Removendo colunas indesejadas
    df['Estacao'] = estacao
    df.drop(columns=[c for c in colunas_remover if c in df.columns], inplace=True)
    dfs.append(df)

# Concatenando tudo em um arquivo .csv
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_csv(os.path.join(pasta, "RS_2009_2023_tempo.csv"), index=False, encoding='utf-8')

print("Arquivo unificado salvo!")

Arquivo unificado salvo!


In [ ]:
clima = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Temperatura/RS_2009_2023_tempo.csv', encoding='utf-8', low_memory=False, decimal=',')

In [ ]:
dff = clima.copy()
clima.head()

,DATA (YYYY-MM-DD),HORA (UTC),"PRECIPITAÇÃO TOTAL, HORÁRIO (mm)","PRESSAO ATMOSFERICA AO NIVEL DA ESTACAO, HORARIA (mB)",RADIACAO GLOBAL (KJ/m²),"TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)",TEMPERATURA DO PONTO DE ORVALHO (°C),"UMIDADE RELATIVA DO AR, HORARIA (%)","VENTO, VELOCIDADE HORARIA (m/s)",Unnamed: 19,Estacao,Data,Hora UTC,RADIACAO GLOBAL (Kj/m²)
0,2009-01-01,00:00,0.0,962.0,-9999,22.8,12.4,52.0,4.5,NaN,CRUZ ALTA,NaN,NaN,NaN
1,2009-01-01,01:00,0.0,962.9,-9999,21.7,13.1,58.0,3.5,NaN,CRUZ ALTA,NaN,NaN,NaN
2,2009-01-01,02:00,0.0,963.1,-9999,20.5,14.4,68.0,3.1,NaN,CRUZ ALTA,NaN,NaN,NaN
3,2009-01-01,03:00,0.0,963.3,-9999,21.5,15.8,70.0,6.2,NaN,CRUZ ALTA,NaN,NaN,NaN
4,2009-01-01,04:00,0.0,962.6,-9999,20.5,16.9,80.0,4.4,NaN,CRUZ ALTA,NaN,NaN,NaN


In [ ]:
# Unificando RADIACAO GLOBAL (KJ/m²) com RADIACAO GLOBAL (Kj/m²) - Puxa o que não tem NaN para nova coluna RADIACAO_GLOBAL
dff['RADIACAO_GLOBAL'] = dff['RADIACAO GLOBAL (KJ/m²)'].combine_first(dff['RADIACAO GLOBAL (Kj/m²)'])
dff['RADIACAO_GLOBAL'] = dff['RADIACAO_GLOBAL'].str.replace(',', '.').astype(float)

# Unificando e ajustando DATA (YYYY-MM-DD) e Data - Puxa o que não tem NaN para nova coluna DATA
dff['DATA'] = dff['DATA (YYYY-MM-DD)'].combine_first(dff['Data'])
dff['DATA'] = dff['DATA'].str.replace('/', '-', regex=False)
# Criando coluna do ano
dff['ANO'] = dff['DATA'].str.split('-').str[0].astype(int)

# Trocando -9999 por NaN
dff.replace(-9999, "NaN", inplace=True)

# Dropando colunas desnecessárias
dff = dff.drop(columns=['RADIACAO GLOBAL (KJ/m²)', 'RADIACAO GLOBAL (Kj/m²)', 'DATA (YYYY-MM-DD)', 'Data', 'HORA (UTC)', 'Hora UTC', 'Unnamed: 19'])

# Ordenando colunas
dff = dff[['Estacao', 'DATA', 'ANO', 'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)', 'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)', 'TEMPERATURA DO PONTO DE ORVALHO (°C)', 'UMIDADE RELATIVA DO AR, HORARIA (%)', 'RADIACAO_GLOBAL', 'VENTO, VELOCIDADE HORARIA (m/s)']]

# Renomeando as Colunas
dff = dff.rename(columns={
    'Estacao': 'Cidade',
    'DATA': 'Data',
    'ANO': 'Ano',
    'PRECIPITAÇÃO TOTAL, HORÁRIO (mm)': 'Precipitacao (mm)',
    'TEMPERATURA DO AR - BULBO SECO, HORARIA (°C)': 'Temp. Ar (°C)',
    'TEMPERATURA DO PONTO DE ORVALHO (°C)': 'Temp. Orvalho (°C)',
    'UMIDADE RELATIVA DO AR, HORARIA (%)': 'Umidade (%)',
    'RADIACAO_GLOBAL': 'Radiacao (KJ/m2)',
    'VENTO, VELOCIDADE HORARIA (m/s)': 'Velocidade Vento (m/s)'})

# Ajustando tipagem
dff = dff.astype({
    'Cidade': 'string',
    'Data': 'datetime64[ns]',
    'Ano': 'int64',
    'Precipitacao (mm)': 'float',
    'Temp. Ar (°C)': 'float',
    'Temp. Orvalho (°C)': 'float',
    'Umidade (%)': 'float',
    'Radiacao (KJ/m2)': 'float',
    'Velocidade Vento (m/s)': 'float'
})

# Tratando nulos com a mediana
mediana = dff.median(numeric_only=True)
dff.fillna(mediana, inplace=True)

dff

,Cidade,Data,Ano,Precipitacao (mm),Temp. Ar (°C),Temp. Orvalho (°C),Umidade (%),Radiacao (KJ/m2),Velocidade Vento (m/s)
0,CRUZ ALTA,2009-01-01,2009,0.0,22.8,12.4,52.0,1009.0,4.5
1,CRUZ ALTA,2009-01-01,2009,0.0,21.7,13.1,58.0,1009.0,3.5
2,CRUZ ALTA,2009-01-01,2009,0.0,20.5,14.4,68.0,1009.0,3.1
3,CRUZ ALTA,2009-01-01,2009,0.0,21.5,15.8,70.0,1009.0,6.2
4,CRUZ ALTA,2009-01-01,2009,0.0,20.5,16.9,80.0,1009.0,4.4
...,...,...,...,...,...,...,...,...,...
5235139,PORTO ALEGRE- BELEM NOVO,2023-12-31,2023,0.0,24.3,15.1,56.0,2409.0,4.0
5235140,PORTO ALEGRE- BELEM NOVO,2023-12-31,2023,0.0,23.9,14.3,55.0,2085.3,4.8
5235141,PORTO ALEGRE- BELEM NOVO,2023-12-31,2023,0.0,23.4,14.1,56.0,1363.9,4.0
5235142,PORTO ALEGRE- BELEM NOVO,2023-12-31,2023,0.0,21.6,14.1,62.0,536.3,4.4


###🔹 Dados Cambiais (USD/BRL)

In [ ]:
# Coletando dados USD/BRL diários de 2009 a 2023

url = "https://olinda.bcb.gov.br/olinda/servico/PTAX/versao/v1/odata/CotacaoDolarPeriodo(dataInicial=@dataInicial,dataFinalCotacao=@dataFinalCotacao)?@dataInicial='01-01-2009'&@dataFinalCotacao='12-31-2023'&$format=json&$select=cotacaoCompra,dataHoraCotacao"

request = requests.request("GET", url).json()
cambio = pd.DataFrame(request['value'])

In [ ]:
# Alterando nome das colunas
cambio.rename(columns={'cotacaoCompra': 'USD/BRL', 'dataHoraCotacao': 'Data'}, inplace=True)

# Formatando a data
cambio['Data'] = cambio['Data'].str.split(' ').str[0]

# Ajustando tipagem
cambio = cambio.astype({'USD/BRL': 'float', 'Data': 'datetime64[ns]'})

# Colunando o ano
cambio['Ano'] = cambio['Data'].dt.year

# Formatando data pt_BR
cambio['Data'] = cambio['Data'].dt.strftime('%d-%m-%Y')

cambio

,USD/BRL,Data,Ano
0,2.3290,02-01-2009,2009
1,2.2772,05-01-2009,2009
2,2.1881,06-01-2009,2009
3,2.2166,07-01-2009,2009
4,2.2675,08-01-2009,2009
...,...,...,...
3759,4.8613,22-12-2023,2023
3760,4.8362,26-12-2023,2023
3761,4.8300,27-12-2023,2023
3762,4.8407,28-12-2023,2023


###🔹 Dados de Comércio Exterior (Market Share exportação)

In [ ]:
# Consolidando todos os arquivos em um único .csv
pasta = "/content/drive/MyDrive/FIAP/Dados Comex"

# Definindo colunas para ignorar
colunas_remover = [
    'CO_VIA',
    'CO_URF',
    'QT_ESTAT',
    'CO_UNID'
]

dfs = []

for arquivo in glob.glob(os.path.join(pasta, "EXP_*.csv")):
    df = pd.read_csv(arquivo, sep=';', encoding='utf-8')
    # Removendo colunas indesejadas
    df.drop(columns=[c for c in colunas_remover if c in df.columns], inplace=True)
    dfs.append(df)

# Concatenando tudo em um arquivo .csv
df_final = pd.concat(dfs, ignore_index=True)
df_final.to_csv(os.path.join(pasta, "RS_2009_2023_comex.csv"), index=False, encoding='utf-8')

print("Arquivo unificado salvo!")

Arquivo unificado salvo!


In [3]:
comex = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Comex/RS_2009_2023_comex.csv', encoding='utf-8', low_memory=False)

In [4]:
ncm = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Comex/NCM.csv', sep=';', encoding="ISO-8859-1")
pais = pd.read_csv('/content/drive/MyDrive/FIAP/Dados Comex/PAIS.csv', sep=';', encoding="ISO-8859-1")

In [8]:
comex_copy = comex.copy()
comex_copy = comex_copy.merge(ncm[['CO_NCM', 'NO_NCM_POR']],on='CO_NCM', how='left')
comex_copy = comex_copy.merge(pais[['CO_PAIS', 'NO_PAIS']],on='CO_PAIS', how='left')
comex_copy = comex_copy.drop(columns='CO_PAIS')
comex_copy = comex_copy[['CO_ANO', 'CO_MES', 'SG_UF_NCM', 'NO_PAIS', 'CO_NCM', 'NO_NCM_POR', 'KG_LIQUIDO']]
comex_copy.head()

,CO_ANO,CO_MES,SG_UF_NCM,NO_PAIS,CO_NCM,NO_NCM_POR,KG_LIQUIDO,VL_FOB,CO_UNID
0,2025,5,SP,México,85369090,"Outros aparelhos para interrupção, etc, para c...",861,44758,11
1,2025,3,RJ,Antígua e Barbuda,9093200,"Sementes de cominho, trituradas ou em pó",1,6,10
2,2025,2,ND,Estados Unidos,90173090,Calibres e semelhantes (instrumento de medida ...,10,881,11
3,2025,2,RJ,Argentina,19053100,"Bolachas e biscoitos, adicionados de edulcorante",66,322,10
4,2025,7,PA,Itália,44123100,"Outras madeiras compensadas, constituídas excl...",24300,31562,16


In [6]:
comex_copy

,CO_ANO,CO_MES,CO_NCM,CO_UNID,SG_UF_NCM,KG_LIQUIDO,VL_FOB,NO_NCM_POR,NO_PAIS
0,2025,5,85369090,11,SP,861,44758,"Outros aparelhos para interrupção, etc, para c...",México
1,2025,3,9093200,10,RJ,1,6,"Sementes de cominho, trituradas ou em pó",Antígua e Barbuda
2,2025,2,90173090,11,ND,10,881,Calibres e semelhantes (instrumento de medida ...,Estados Unidos
3,2025,2,19053100,10,RJ,66,322,"Bolachas e biscoitos, adicionados de edulcorante",Argentina
4,2025,7,44123100,16,PA,24300,31562,"Outras madeiras compensadas, constituídas excl...",Itália
...,...,...,...,...,...,...,...,...,...
20532584,2009,9,87088000,11,SP,15,292,Amortecedores de suspensão para tratores e veí...,Angola
20532585,2009,9,96061000,10,RS,86,1222,Botões de pressão e suas partes,Honduras
20532586,2009,9,94017900,11,RJ,28,204,Outros assentos com armação de metal,Angola
20532587,2009,9,99980101,10,RJ,10750,21978,Consumo de bordo - combustíveis e lubrificante...,Reino Unido


In [ ]:
# TODA A BASE DE DADOS É REFERENTE A EXPORTAÇÃO
# NCM = VINHOS DE MESA TRANQUILOS REPRESENTAM X% DAS EXPORTAÇÕES DO BRASIL E X% DO RS (VALOR FOB / INFO DE MERCADO)
# PAIS = MAIORES IMPORTADORES (OPORTUNIDADES DE NEGÓCIOS/DEMANDA REPRIMIDA)
# ESTADO = PARTICIPAÇÃO % DA VINÍCULA PERANTE RS E BRASIL (KG)
# SAZONALIDADE(MÊS) DE EXPORTAÇÃO DE VINHO


# ncms referentes a vinhos tranquilos
ncms_vinho = [22042211, 22042219, 22042910, 22042100,
              22042900, 22042911, 22042919]

## Market Share:
#falando em FOB (economia) - EXPORTAÇÕES são registradas em valor FOB na balança comercial para medir o impacto econômico (Free On Board = produto + transporte interno + serviços portuários e aduaneiros).
valor_fob_brasil = comex_copy['VL_FOB'].sum()
valor_fob_vinhos_br = comex_copy[comex_copy['CO_NCM'].isin(ncms_vinho)]['VL_FOB'].sum() #Os vinhos tranquilos representam X% do valor FOB das exportações brasileiras. (valor_fob_vinhos_br/valor_fob_brasil)
valor_fob_vinhos_rs = comex_copy[(comex_copy['CO_NCM'].isin(ncms_vinho)) & (comex_copy['SG_UF_NCM'] == 'RS')]['VL_FOB'].sum() #O RS responde por X% do valor FOB das exportações brasileiras de vinhos tranquilos, ocupando a Xª posição entre os estados exportadores. (valor_fob_vinhos_rs/valor_fob_vinhos_br)

#falando em KGs
valor_kg_vinhos_br = comex_copy[comex_copy['CO_NCM'].isin(ncms_vinho)]['KG_LIQUIDO'].sum() # Total de vinhos tranquilos exportados pelo Brasil em kg
valor_kg_vinhos_rs = comex_copy[(comex_copy['CO_NCM'].isin(ncms_vinho)) & (comex_copy['SG_UF_NCM'] == 'RS')]['KG_LIQUIDO'].sum() # % do RS sobre o total brasileiro exportado de vinhos em kg (valor_fob_vinhos_rs/valor_fob_vinhos_br)
# valor_kg_vinicula/valor_kg_RS e BR

#falando em países parceiros
# Os países 80/20 nas exportações de vinhos (comexstat), quais as oportunidades?


###🔹 Dados de Avaliações

###🔹 Dados Demograficos


###🔹 Dados Produção Agricola Municipal (PAM)
